# StyleGAN2-ada

This code runs the Nvidia Labs implementation of StyleGan2-ada on Pytorch 2 from their StyleGAN3 repository as it has support for modern versons of Cuda and Python. You can find that repository here: https://github.com/NVlabs/stylegan3

We ran StyleGAN2-ada over StyleGAN3 to stay in realistic memory contraints.

You can find this code and the other code we have written in relation to this project here: https://github.com/MohitAgrawal404/DCGAN_Sprite_Generation/tree/main


This is to mount your Google Drive. This will just be at the root.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install Stylegan2-ada-pytorch prereqs

In [2]:
import torch
import torchvision
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: imageio-ffmpeg
    Found existing installation: imageio-ffmpeg 0.6.0
    Uninstalling imageio-ffmpeg-0.6.0:
      Successfully uninstalled imageio-ffmpeg-0.6.0


StyleGAN2-ada code from StyleGAN3 repository

In [3]:
!git clone https://github.com/NVlabs/stylegan3.git

Cloning into 'stylegan3'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 212 (delta 99), reused 90 (delta 90), pack-reused 49 (from 1)
Receiving objects: 100% (212/212), 4.16 MiB | 10.47 MiB/s, done.
Resolving deltas: 100% (108/108), done.


This loads the dataset from our github. Make sure to change 'which_dataset' to either pokemon or human based on which dataset you want to run.

If you have already downloaded the dataset into the root of your project then you can skip this OR if you aren't sure then just type 'A' when it prompts you to replace something which will replace the whole folder (you will need to double click where it prompts you).


In [4]:
!mkdir /content/stylegan3/datasets
!cd /content/stylegan3/datasets

In [5]:
which_dataset = "pokemon"
folder_name = f"{which_dataset}_ada_sprites"
zip_file = f"{folder_name}.zip"
url = f"https://raw.githubusercontent.com/MohitAgrawal404/DCGAN_Sprite_Generation/main/{zip_file}"

!wget "{url}"

!unzip "{zip_file}" -d /content/stylegan3/datasets/

!rm "{zip_file}"



--2025-12-06 18:43:16--  https://raw.githubusercontent.com/MohitAgrawal404/DCGAN_Sprite_Generation/main/pokemon_ada_sprites.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5487916 (5.2M) [application/zip]
Saving to: ‘pokemon_ada_sprites.zip’

pokemon_ada_sprites 100%[===================>]   5.23M  --.-KB/s    in 0.07s   

2025-12-06 18:43:17 (70.6 MB/s) - ‘pokemon_ada_sprites.zip’ saved [5487916/5487916]

Archive:  pokemon_ada_sprites.zip
   creating: /content/stylegan3/datasets/pokemon_ada_like/
  inflating: /content/stylegan3/datasets/pokemon_ada_like/1-gen3_e-frame2.png  
  inflating: /content/stylegan3/datasets/pokemon_ada_like/1-gen3_e.png  
  inflating: /content/stylegan3/datasets/pokemon_ada_like/1-gen3_fl.png  
  inflating: /content/stylegan3/dat

Getting dataset ready for use by StyleGAN2-ada

In [6]:
source_dir = f"/content/stylegan3/datasets/{which_dataset}_ada_like"
dest_zip = f"/content/stylegan3/datasets/{which_dataset}dataset.zip"

!python /content/stylegan3/dataset_tool.py --source="{source_dir}" --dest="{dest_zip}"

  0% 0/4086 [00:00<?, ?it/s]/content/stylegan3/dataset_tool.py:441: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = PIL.Image.fromarray(img, { 1: 'L', 3: 'RGB' }[channels])
100% 4086/4086 [00:02<00:00, 1790.71it/s]


There are a lot of files that it spits out so we are just going to put it in your main google drive for you to view easily at the directory listed below. Don't change the location though.

In [7]:
!mkdir /content/drive/MyDrive/StyleGAN2-ada-output

mkdir: cannot create directory ‘/content/drive/MyDrive/StyleGAN2-ada-output’: File exists


In [8]:
target_dir = f"/content/drive/MyDrive/StyleGAN2-ada-output/{which_dataset}"
!mkdir -p "{target_dir}"

So there are some issues with compatability because of the Python version and some of the libraries being used so this is the fix to basically override the file with the old implementation. This is more an issue with colab being difficult to downgrade versions of things.

In [9]:
import os
import sys
import torch


os.system('pip install "setuptools<70.0.0" ninja')


custom_ops_content = r"""
import sys
import os
import torch
import torch.utils.cpp_extension
import importlib

verbose = True

# We add **kwargs to accept (and ignore) whatever broken flags the caller sends
def get_plugin(module_name, sources, headers=(), source_dir=None, **kwargs):
    if source_dir is None:
        source_dir = os.path.dirname(__file__)

    # Define paths
    sources = [os.path.join(source_dir, src) for src in sources]
    headers = [os.path.join(source_dir, hdr) for hdr in headers]

    print(f'\n[DEBUG] Compiling {module_name}...')

    # We ignore the 'extra_cuda_cflags' passed in kwargs and use our own
    # This fixes the specific crash you just saw.
    return torch.utils.cpp_extension.load(
        name=module_name,
        sources=sources,
        extra_cflags=['-O3'],
        extra_cuda_cflags=['-O3'],
        verbose=True
    )
"""

target_path = "/content/stylegan3/torch_utils/custom_ops.py"
print(f"Overwriting {target_path}...")
with open(target_path, "w") as f:
    f.write(custom_ops_content)


import shutil
cache_dir = os.path.expanduser('~/.cache/torch_extensions')
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
    print("Cache cleared.")

print("Compiling plugns")

sys.path.append("/content/stylegan3")
import torch_utils.custom_ops
import importlib
importlib.reload(torch_utils.custom_ops)

try:
    from torch_utils.ops import bias_act
    from torch_utils.ops import upfirdn2d

    # Triggering the build
    if bias_act._init():
        print("bias_act compiled")

    if upfirdn2d._init():
        print("upfirdn2d compiled")

    print("\n Ready to train.")

except Exception as e:
    print(f"\nERROR: {e}")

Overwriting /content/stylegan3/torch_utils/custom_ops.py...
Compiling plugns

[DEBUG] Compiling bias_act_plugin...
bias_act compiled

[DEBUG] Compiling upfirdn2d_plugin...
upfirdn2d compiled

 Ready to train.


This cell block will train StyleGAN. You can have it keep running for as long as you want and then just abort it. View the image in the folder it creates in your drive. Find the one that is a png that starts with fake.

In [10]:
cd /content/stylegan3/

/content/stylegan3


In [12]:
!python train.py \
  --outdir="{target_dir}" \
  --cfg=stylegan2 \
  --data=/content/stylegan3/datasets/pokemondataset.zip \
  --gpus=1 \
  --batch=32 \
  --gamma=10 \
  --mirror=1 \
  --snap=5 \
  --metrics=none # --resume=/content/drive/MyDrive/StyleGAN2-ada-output/pokemon/00002-stylegan2-pokemondataset-gpus1-batch32-gamma10/network-snapshot-000160.pkl


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan2.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "channel_base": 32768,
    "channel_max": 512,
    "fused_modconv_default": "inference_only"
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 10.0,
    "style_mixing_prob": 0.9,
    "pl_weight": 2,
    "pl_no_weight_gra